In [9]:
import numpy as np
from tqdm import tqdm
import os,json
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from keras.optimizers import Adam

Using TensorFlow backend.


In [29]:
import pandas as pd
import pickle as pk
min_count = 10

In [31]:
db = pd.read_csv('./data/news/News_result.csv')
chars = {}
#chars = dict()
for indexs, a in tqdm(db[0:2].iterrows()):
    #print(a['content'])
    for w in a['content']: # 纯文本，不用分词
        chars[w] = chars.get(w,0) + 1
    for w in a['title']: # 纯文本，不用分词
        chars[w] = chars.get(w,0) + 1
chars = {i:j for i,j in chars.items() if j >= min_count}

2it [00:00, 666.71it/s]


In [35]:
id2char = {i+4:j for i,j in enumerate(chars)}
char2id = {j:i for i,j in id2char.items()}

In [37]:
len(chars)

58